# EfficientNet Version 6 (RGB)
This notebook trains the 6th version of the EfficientNet model, which modifies version 5 to train on the RGB dataset (with the second iteration of pre-processing).

## 1. Mount Google Drive (Colab Only)
This section connects to Google Drive to access the dataset.

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

DATASET_PATH = '/content/drive/My Drive/50.021 Artificial Intelligence Group Assignment'

%cd "/content/drive/My Drive/50.021 Artificial Intelligence Group Assignment"

import os
assert os.path.exists(DATASET_PATH), "[!] Dataset path does not exist. Please check the path."

Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/12m4fX9cAr1HVdx1AB1mgPF26R9bqG4mk/50.021 Artificial Intelligence Group Assignment


In [ ]:
%%capture
def is_running_in_colab():
    try:
        import google.colab
        return True
    except ImportError:
        return False

if is_running_in_colab():
  %pip install lightning polars
  %pip install icecream rich tqdm

## 2. Import Libraries
Imports all required packages including PyTorch, Lightning, and data processing tools.

In [ ]:
# Imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import polars as pl
import numpy as np
import lightning as L
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
from datetime import datetime

# Model imports
from torchvision.models import (
    efficientnet_v2_s, EfficientNet_V2_S_Weights,
    efficientnet_v2_m, EfficientNet_V2_M_Weights,
    efficientnet_v2_l, EfficientNet_V2_L_Weights
)
from torchvision.transforms import v2

# Metrics imports
from torchmetrics import (
    Accuracy,
    F1Score,
    AUROC,
    ConfusionMatrix,
    Precision,
    Recall
)

# Lightning modules and callbacks
from lightning.pytorch.callbacks import (
    ModelCheckpoint,
    EarlyStopping,
    LearningRateMonitor
)
from pytorch_lightning.loggers import TensorBoardLogger

# Set random seeds for reproducibility
import random
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

In [ ]:
print("GPU available:", torch.cuda.is_available())

GPU available: True


## 3. Define Dataset Classes

In [ ]:
class ImageDataset(torch.utils.data.Dataset):
    """Dataset for loading and preprocessing garbage classification images."""

    def __init__(self, dataframe: pl.DataFrame, training=False, img_size=224):
        super().__init__()
        # Use the image_path column directly from the dataframe as it now contains full paths
        # to the preprocessed RGB images
        self.image_path = dataframe['image_path'].to_numpy().squeeze()
        self.garbage_type = dataframe.select('label').to_numpy().squeeze()
        self.garbage_to_idx = {garbage: i for i, garbage in enumerate(np.unique(self.garbage_type))}
        self.training = training
        self.img_size = img_size

        # Enhanced augmentation for training - can be simpler now since images are already preprocessed
        self.train_transforms = v2.Compose([
            v2.RandomHorizontalFlip(p=0.5),
            v2.RandomVerticalFlip(p=0.3),
            v2.RandomRotation(degrees=10),
            # Reduced augmentation as images are already preprocessed and augmented
        ])

        # Base transforms for all images
        self.transforms = v2.Compose([
            v2.ToDtype(torch.float32, scale=True),
        ])

    def __len__(self):
        return len(self.image_path)

    def __getitem__(self, idx):
        image_path = self.image_path[idx]

        try:
            # Load the image - now these are RGB processed images
            image = cv2.imread(image_path)
            if image is None:
                raise ValueError(f"Failed to load image at {image_path}")

            # Convert BGR to RGB (cv2 loads as BGR)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            # Resize to the target size
            image = cv2.resize(image, (self.img_size, self.img_size))

            # Convert to tensor and reorder dimensions to [C, H, W]
            image = torch.from_numpy(image).float().permute(2, 0, 1)

            # Normalize to [0, 1]
            image = image / 255.0

            # Apply augmentations if in training mode
            if self.training:
                image = self.train_transforms(image)

            # Apply basic transforms
            image = self.transforms(image)

            # Get class label
            garbage = self.garbage_to_idx[self.garbage_type[idx]]

            return image, torch.tensor(garbage, dtype=torch.long)

        except Exception as e:
            print(f"Error loading image {image_path}: {e}")
            raise

In [ ]:
class GarbageClassificationData(L.LightningDataModule):
    """Data module for garbage classification dataset."""

    def __init__(self, ws_root: Path = Path("."), batch_size=32, num_workers=0, img_size=224):
        super().__init__()

        self.batch_size = batch_size
        self.img_size = img_size

        # Updated paths to use the new RGB preprocessed images
        rgb_base_path = Path("/content/drive/My Drive/50.021 Artificial Intelligence Group Assignment/RGB_preprocessed_images")

        # Load datasets using the new CSV files
        self.train_ds = ImageDataset(
            pl.read_csv(rgb_base_path / 'train.csv'),
            training=True,
            img_size=self.img_size
        )
        self.val_ds = ImageDataset(
            pl.read_csv(rgb_base_path / 'validation.csv'),
            img_size=self.img_size
        )
        self.test_ds = ImageDataset(
            pl.read_csv(rgb_base_path / 'test.csv'),
            img_size=self.img_size
        )

        # Get class info
        self.n_classes = len(self.train_ds.garbage_to_idx)
        self.idx_to_garbage = {v: k for k, v in self.train_ds.garbage_to_idx.items()}

        # Print dataset statistics
        print(f"Number of training samples: {len(self.train_ds)}")
        print(f"Number of validation samples: {len(self.val_ds)}")
        print(f"Number of test samples: {len(self.test_ds)}")
        print(f"Number of classes: {self.n_classes}")
        print(f"Classes: {self.idx_to_garbage}")

        # DataLoader settings
        self.dataloader_extras = dict(
            num_workers=num_workers,
            pin_memory=True,
            persistent_workers=num_workers > 0
        )

    def train_dataloader(self):
        return torch.utils.data.DataLoader(
            self.train_ds,
            batch_size=self.batch_size,
            shuffle=True,
            **self.dataloader_extras
        )

    def val_dataloader(self):
        return torch.utils.data.DataLoader(
            self.val_ds,
            batch_size=self.batch_size*2,
            **self.dataloader_extras
        )

    def test_dataloader(self):
        return torch.utils.data.DataLoader(
            self.test_ds,
            batch_size=self.batch_size*2,
            **self.dataloader_extras
        )

## 4. Base Classifier

In [ ]:
class BaseGarbageClassifier(L.LightningModule):
    """Base class for all garbage classification models."""

    def __init__(self, n_classes, learning_rate=1e-3):
        super().__init__()
        self.n_classes = n_classes
        self.learning_rate = learning_rate
        self.criterion = nn.CrossEntropyLoss()

        # Initialize metrics
        self.train_acc = Accuracy(task="multiclass", num_classes=n_classes)
        self.val_acc = Accuracy(task="multiclass", num_classes=n_classes)

        # Test metrics
        self.test_accuracy = Accuracy(task="multiclass", num_classes=n_classes)
        self.test_precision = Precision(task="multiclass", num_classes=n_classes, average='macro')
        self.test_recall = Recall(task="multiclass", num_classes=n_classes, average='macro')
        self.test_f1 = F1Score(task="multiclass", num_classes=n_classes, average='macro')
        self.test_auroc = AUROC(task="multiclass", num_classes=n_classes)
        self.test_confusion_matrix = ConfusionMatrix(task="multiclass", num_classes=n_classes)

        # Save hyperparameters for checkpointing
        self.save_hyperparameters()

    def forward(self, x):
        """Forward pass of the model. To be implemented by subclasses."""
        raise NotImplementedError("Subclasses must implement forward()")

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_pred = self(x)
        loss = self.criterion(y_pred, y)

        # Calculate and log metrics
        preds = torch.argmax(y_pred, dim=1)
        acc = self.train_acc(preds, y)

        # Log metrics
        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True)
        self.log('train_acc', acc, on_step=True, on_epoch=True, prog_bar=True)

        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_pred = self(x)
        loss = self.criterion(y_pred, y)

        # Calculate metrics
        preds = torch.argmax(y_pred, dim=1)
        acc = self.val_acc(preds, y)

        # Log metrics
        self.log('val_loss', loss, on_epoch=True, prog_bar=True)
        self.log('val_acc', acc, on_epoch=True, prog_bar=True)

        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_pred = self(x)
        loss = self.criterion(y_pred, y)

        # Calculate predictions
        preds = torch.argmax(y_pred, dim=1)
        probs = torch.softmax(y_pred, dim=1)

        # Update metrics
        self.test_accuracy(preds, y)
        self.test_precision(preds, y)
        self.test_recall(preds, y)
        self.test_f1(preds, y)
        self.test_auroc(probs, y)
        self.test_confusion_matrix(preds, y)

        # Log metrics
        self.log('test_loss', loss, on_epoch=True)

        return loss

    def on_test_epoch_end(self):
        # Compute and log final metrics
        accuracy = self.test_accuracy.compute()
        precision = self.test_precision.compute()
        recall = self.test_recall.compute()
        f1_score = self.test_f1.compute()
        auroc = self.test_auroc.compute()
        conf_mat = self.test_confusion_matrix.compute().cpu().numpy()

        # Log metrics
        self.log('test_accuracy', accuracy)
        self.log('test_precision', precision)
        self.log('test_recall', recall)
        self.log('test_f1', f1_score)
        self.log('test_auroc', auroc)

        # Print detailed metrics
        print("\n--- Test Metrics ---")
        print(f"Accuracy: {accuracy:.4f}")
        print(f"Precision: {precision:.4f}")
        print(f"Recall: {recall:.4f}")
        print(f"F1 Score: {f1_score:.4f}")
        print(f"AUROC: {auroc:.4f}")

        # Visualize Confusion Matrix
        plt.figure(figsize=(12, 10))
        sns.heatmap(conf_mat, annot=True, fmt='d', cmap='Blues')
        plt.title('Test Confusion Matrix')
        plt.xlabel('Predicted Label')
        plt.ylabel('True Label')
        plt.tight_layout()
        plt.savefig(f"confusion_matrix_{self.__class__.__name__}.png")
        plt.show()

        # Reset metrics
        self.test_accuracy.reset()
        self.test_precision.reset()
        self.test_recall.reset()
        self.test_f1.reset()
        self.test_auroc.reset()
        self.test_confusion_matrix.reset()

    def configure_optimizers(self):
        """Configure the optimizer and learning rate scheduler."""
        optimizer = torch.optim.AdamW(
            self.parameters(),
            lr=self.learning_rate,
            weight_decay=1e-4
        )

        # Learning rate scheduler
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer,
            mode='min',
            factor=0.5,
            patience=2,
            min_lr=1e-6
        )

        return {
            "optimizer": optimizer,
            "lr_scheduler": {
                "scheduler": scheduler,
                "monitor": "val_loss",
                "interval": "epoch",
                "frequency": 1
            }
        }

## 5. Training Utilities / Functions

In [ ]:
def train_and_evaluate_model(model_class, data_module, model_name, max_epochs=10, learning_rate=1e-3):
    """Train and evaluate a model without checkpointing."""

    # Create timestamp for run identification
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    run_name = f"{model_name}_{timestamp}"

    # Create logger (for metrics, not checkpoints)
    logger = TensorBoardLogger(
        save_dir='logs',
        name=run_name,
        log_graph=False  # Disable graph logging to reduce overhead
    )

    # Create callbacks (no checkpoint callback)
    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=5,
        mode='min'
    )

    lr_monitor = LearningRateMonitor(logging_interval='epoch')

    # Initialize trainer without checkpoint callback
    trainer = L.Trainer(
        max_epochs=max_epochs,
        accelerator='gpu' if torch.cuda.is_available() else 'cpu',
        precision="16-mixed" if torch.cuda.is_available() else "32-true",
        callbacks=[early_stopping, lr_monitor],
        logger=logger,
        log_every_n_steps=10,
        enable_checkpointing=False  # Disable checkpointing
    )

    # Initialize model
    model = model_class(n_classes=data_module.n_classes, learning_rate=learning_rate)

    # Train the model
    print(f"\nTraining {model_name}...")
    trainer.fit(model=model, datamodule=data_module)

    # Test the model
    print(f"\nEvaluating {model_name}...")
    trainer.test(model=model, datamodule=data_module)

    # Save the final model (after training and testing)
    try:
        model_save_path = f"{model_name}_{timestamp}_final.pt"
        # Save only the model state dict, not the entire checkpoint
        torch.save(model.state_dict(), model_save_path)
        print(f"Model saved to {model_save_path}")
    except Exception as e:
        print(f"Warning: Could not save model due to: {e}")
        print("Continuing without saving...")

    # Return the trained model and its metrics
    return model

## Method 6: Modifying Method 5 to train on RGB Dataset

In [ ]:
from torch.optim.lr_scheduler import OneCycleLR

In [ ]:
class EfficientNetV2MOptimized(L.LightningModule):
    """
    Enhanced EfficientNetV2-M with Optimized Regularization:
    1. Learning rate scheduling
    2. Progressive unfreezing
    3. Advanced regularization techniques
    4. Mixup data augmentation
    5. Optimized training process
    """

    def __init__(self, n_classes, learning_rate=3e-4, weight_decay=1e-4):
        super().__init__()
        self.n_classes = n_classes
        self.learning_rate = learning_rate
        self.weight_decay = weight_decay
        self.mixup_alpha = 0.2  # Parameter for mixup augmentation

        # Load pre-trained EfficientNetV2-M model
        self.model = efficientnet_v2_m(weights=EfficientNet_V2_M_Weights.DEFAULT)

        # IMPORTANT: No need to modify the first conv layer for RGB inputs
        # since the model already expects 3 channels by default
        # REMOVE these lines as they were converting the model to accept grayscale
        # original_conv = self.model.features[0][0]
        # new_conv = nn.Conv2d(...)
        # self.model.features[0][0] = new_conv

        # Initialize frozen layer flags for progressive unfreezing
        self.unfreeze_stage = 0

        # Initially freeze all feature extraction layers
        for i in range(len(self.model.features)):
            for param in self.model.features[i].parameters():
                param.requires_grad = False

        # Get the number of features in the final layer
        in_features = self.model.classifier[1].in_features

        # Replace classifier with enhanced MLP head
        self.model.classifier = nn.Sequential(
            nn.Dropout(0.4),  # Increased dropout rate
            nn.Linear(in_features, 1024),
            nn.BatchNorm1d(1024),
            nn.SiLU(),  # SiLU/Swish activation (better than ReLU)
            nn.Dropout(0.3),
            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.SiLU(),
            nn.Dropout(0.2),
            nn.Linear(512, n_classes)
        )

        # Use label smoothing cross entropy
        self.criterion = nn.CrossEntropyLoss(label_smoothing=0.1)

        # Initialize metrics
        self.train_acc = Accuracy(task="multiclass", num_classes=n_classes)
        self.val_acc = Accuracy(task="multiclass", num_classes=n_classes)

        # Test metrics
        self.test_accuracy = Accuracy(task="multiclass", num_classes=n_classes)
        self.test_precision = Precision(task="multiclass", num_classes=n_classes, average='macro')
        self.test_recall = Recall(task="multiclass", num_classes=n_classes, average='macro')
        self.test_f1 = F1Score(task="multiclass", num_classes=n_classes, average='macro')
        self.test_auroc = AUROC(task="multiclass", num_classes=n_classes)
        self.test_confusion_matrix = ConfusionMatrix(task="multiclass", num_classes=n_classes)

        # Save hyperparameters
        self.save_hyperparameters()

    def mixup_data(self, x, y):
        """Apply mixup augmentation to the batch."""
        if self.training and self.mixup_alpha > 0:
            lam = np.random.beta(self.mixup_alpha, self.mixup_alpha)
            batch_size = x.size()[0]
            index = torch.randperm(batch_size).to(x.device)

            mixed_x = lam * x + (1 - lam) * x[index, :]
            y_a, y_b = y, y[index]
            return mixed_x, y_a, y_b, lam
        else:
            return x, y, y, 1.0

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch

        # Apply mixup if training
        if self.training:
            x, y_a, y_b, lam = self.mixup_data(x, y)
            y_pred = self(x)
            loss = lam * self.criterion(y_pred, y_a) + (1 - lam) * self.criterion(y_pred, y_b)
        else:
            y_pred = self(x)
            loss = self.criterion(y_pred, y)

        # Calculate and log metrics
        preds = torch.argmax(y_pred, dim=1)
        acc = self.train_acc(preds, y)

        # Log metrics
        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True)
        self.log('train_acc', acc, on_step=True, on_epoch=True, prog_bar=True)

        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_pred = self(x)
        loss = self.criterion(y_pred, y)

        # Calculate metrics
        preds = torch.argmax(y_pred, dim=1)
        acc = self.val_acc(preds, y)

        # Log metrics
        self.log('val_loss', loss, on_epoch=True, prog_bar=True)
        self.log('val_acc', acc, on_epoch=True, prog_bar=True)

        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_pred = self(x)
        loss = self.criterion(y_pred, y)

        # Calculate predictions
        preds = torch.argmax(y_pred, dim=1)
        probs = torch.softmax(y_pred, dim=1)

        # Update metrics
        self.test_accuracy(preds, y)
        self.test_precision(preds, y)
        self.test_recall(preds, y)
        self.test_f1(preds, y)
        self.test_auroc(probs, y)
        self.test_confusion_matrix(preds, y)

        # Log metrics
        self.log('test_loss', loss, on_epoch=True)

        return loss

    def on_test_epoch_end(self):
        # Compute and log final metrics
        accuracy = self.test_accuracy.compute()
        precision = self.test_precision.compute()
        recall = self.test_recall.compute()
        f1_score = self.test_f1.compute()
        auroc = self.test_auroc.compute()
        conf_mat = self.test_confusion_matrix.compute().cpu().numpy()

        # Store metrics in self for access after training
        self.final_metrics = {
            'accuracy': accuracy.item(),
            'precision': precision.item(),
            'recall': recall.item(),
            'f1': f1_score.item(),
            'auroc': auroc.item()
        }

        # Print detailed metrics
        print("\n--- Test Metrics ---")
        print(f"Accuracy: {accuracy:.4f}")
        print(f"Precision: {precision:.4f}")
        print(f"Recall: {recall:.4f}")
        print(f"F1 Score: {f1_score:.4f}")
        print(f"AUROC: {auroc:.4f}")

        # Reset metrics
        self.test_accuracy.reset()
        self.test_precision.reset()
        self.test_recall.reset()
        self.test_f1.reset()
        self.test_auroc.reset()
        self.test_confusion_matrix.reset()

    def on_train_epoch_start(self):
        """
        Progressive unfreezing of layers based on training epoch.
        """
        # Unfreeze more layers as training progresses
        total_blocks = len(self.model.features)

        if self.current_epoch == 3 and self.unfreeze_stage == 0:
            # Unfreeze the last 30% of feature layers after 3 epochs
            unfreeze_from = int(total_blocks * 0.7)
            for i in range(unfreeze_from, total_blocks):
                for param in self.model.features[i].parameters():
                    param.requires_grad = True
            print(f"Unfreezing layers from {unfreeze_from} to {total_blocks-1}")
            self.unfreeze_stage = 1

        elif self.current_epoch == 6 and self.unfreeze_stage == 1:
            # Unfreeze more layers after 6 epochs
            unfreeze_from = int(total_blocks * 0.4)
            for i in range(unfreeze_from, total_blocks):
                for param in self.model.features[i].parameters():
                    param.requires_grad = True
            print(f"Unfreezing layers from {unfreeze_from} to {total_blocks-1}")
            self.unfreeze_stage = 2

        elif self.current_epoch == 9 and self.unfreeze_stage == 2:
            # Unfreeze all layers after 9 epochs
            for i in range(total_blocks):
                for param in self.model.features[i].parameters():
                    param.requires_grad = True
            print("Unfreezing all layers")
            self.unfreeze_stage = 3

    def configure_optimizers(self):
        """
        Configure optimizer with weight decay and learning rate scheduler.
        """
        # Create optimizer with weight decay
        optimizer = torch.optim.AdamW(
            self.parameters(),
            lr=self.learning_rate,
            weight_decay=self.weight_decay
        )

        # OneCycleLR scheduler
        scheduler = {
            "scheduler": OneCycleLR(
                optimizer,
                max_lr=self.learning_rate,
                total_steps=self.trainer.estimated_stepping_batches,
                pct_start=0.1,  # 10% warmup
                div_factor=25,  # initial_lr = max_lr/25
                final_div_factor=1000,  # min_lr = initial_lr/1000
            ),
            "interval": "step",
            "frequency": 1
        }

        return {"optimizer": optimizer, "lr_scheduler": scheduler}

## Training + Evaluation of methods

In [ ]:
# Prepare data module (shared between models)
data_module = GarbageClassificationData(
    batch_size=16,
    num_workers=4,
    img_size=224
)

Number of training samples: 12670
Number of validation samples: 1091
Number of test samples: 1090
Number of classes: 8
Classes: {0: 'Glass', 1: 'Hazardous Waste', 2: 'Metal', 3: 'Organic Waste', 4: 'Paper & Cardboard', 5: 'Plastic', 6: 'Textiles', 7: 'Trash'}


### Method 6 Training

In [ ]:
# Model 5
print("\n=== Training Model 5: Enhanced EfficientNetV2-M with Optimized Regularization ===")
model5 = train_and_evaluate_model(
  model_name="EfficientNetV2MOptimized",
  model_class=EfficientNetV2MOptimized,
  data_module=data_module,
  max_epochs=20,
  learning_rate=3e-4
)

INFO:pytorch_lightning.utilities.rank_zero:Using 16bit Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs



=== Training Model 5: Enhanced EfficientNetV2-M with Optimized Regularization ===


INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loading `train_dataloader` to estimate number of stepping batches.



Training EfficientNetV2MOptimized...


INFO: 
  | Name                  | Type                      | Params | Mode 
----------------------------------------------------------------------------
0 | model                 | EfficientNet              | 54.7 M | train
1 | criterion             | CrossEntropyLoss          | 0      | train
2 | train_acc             | MulticlassAccuracy        | 0      | train
3 | val_acc               | MulticlassAccuracy        | 0      | train
4 | test_accuracy         | MulticlassAccuracy        | 0      | train
5 | test_precision        | MulticlassPrecision       | 0      | train
6 | test_recall           | MulticlassRecall          | 0      | train
7 | test_f1               | MulticlassF1Score         | 0      | train
8 | test_auroc            | MulticlassAUROC           | 0      | train
9 | test_confusion_matrix | MulticlassConfusionMatrix | 0      | train
----------------------------------------------------------------------------
1.8 M     Trainable params
52.9 M    Non-trainable params


...

Testing: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/torchmetrics/utilities/prints.py:43: UserWarning: No negative samples in targets, false positive value should be meaningless. Returning zero tensor in false positive score
  warnings.warn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torchmetrics/utilities/prints.py:43: UserWarning: No positive samples in targets, true positive value should be meaningless. Returning zero tensor in true positive score
  warnings.warn(*args, **kwargs)



--- Test Metrics ---
Accuracy: 0.9651
Precision: 0.9650
Recall: 0.9649
F1 Score: 0.9646
AUROC: 0.9972


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │    0.5643470287322998     │
└───────────────────────────┴───────────────────────────┘

Model saved to EfficientNetV2MOptimized_20250417_072521_final.pt


In [ ]:
torch.save(model5, "saved_efficientnet/efficientnetv2-method5-RGB.pth")

## Load saved model and test with images

In [ ]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [ ]:
model = torch.load("saved_efficientnet/efficientnetv2-method5-RGB.pth", map_location=device, weights_only=False)
model.to(device)
model.eval()

EfficientNetV2MOptimized(
  (model): EfficientNet(
    (features): Sequential(
      (0): Conv2dNormActivation(
        (0): Conv2d(3, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
        (2): SiLU(inplace=True)
      )
      (1): Sequential(
        (0): FusedMBConv(
          (block): Sequential(
            (0): Conv2dNormActivation(
              (0): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
              (1): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
              (2): SiLU(inplace=True)
            )
          )
          (stochastic_depth): StochasticDepth(p=0.0, mode=row)
        )
        (1): FusedMBConv(
          (block): Sequential(
            (0): Conv2dNormActivation(
              (0): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
              (1): B

In [ ]:
data_module = GarbageClassificationData()
test_dataset = data_module.test_ds

Number of training samples: 12670
Number of validation samples: 1091
Number of test samples: 1090
Number of classes: 8
Classes: {0: 'Glass', 1: 'Hazardous Waste', 2: 'Metal', 3: 'Organic Waste', 4: 'Paper & Cardboard', 5: 'Plastic', 6: 'Textiles', 7: 'Trash'}


In [ ]:
import random
import matplotlib.pyplot as plt
import torch

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Pick random test samples
num_samples = 20
indices = random.sample(range(len(test_dataset)), num_samples)

for i in indices:
    image, label = test_dataset[i]
    image_input = image.unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(image_input)
        pred = torch.argmax(output, dim=1).item()

    image_np = image.permute(1, 2, 0).cpu().numpy()

    plt.imshow(image_np)
    plt.title(f"True: {data_module.idx_to_garbage[label.item()]}, Predicted: {data_module.idx_to_garbage[pred]}")
    plt.axis('off')
    plt.show()

Output hidden; open in https://colab.research.google.com to view.